In [1]:
import os

In [2]:
pwd

'/Users/salonisahal/Desktop/Chicken-Disease-Classification-Project/research'

In [3]:
os.chdir("../")

In [4]:
pwd

'/Users/salonisahal/Desktop/Chicken-Disease-Classification-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directory

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])
    

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)
        create_directory([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config= PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callbacks_config
    

In [10]:
import urllib.request as request
import zipfile
import tensorflow as tf
from zipfile import ZipFile
import time

In [13]:
class PrepareCallbacks:
    def __init__(self,config: PrepareCallbacksConfig):
        self.config=config

    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(
            str(self.config.tensorboard_root_log_dir),
            f"tb_logs_at_{timestamp}",
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
        

In [14]:
try:
    config=ConfigurationManager()
    prepare_callbacks_config=config.get_prepare_callback_config()
    prepare_callbacks=PrepareCallbacks(config=prepare_callbacks_config)
    callback_list=prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2025-04-04 21:03:02,678: INFO: common:yaml file:config/config.yaml loaded Successfully]
[2025-04-04 21:03:02,681: INFO: common:yaml file:params.yaml loaded Successfully]
[2025-04-04 21:03:02,682: INFO: common:Directory artifacts created successfully.]
[2025-04-04 21:03:02,683: INFO: common:Directory artifacts/prepare_callbacks/checkpoint_dir created successfully.]
[2025-04-04 21:03:02,684: INFO: common:Directory artifacts/prepare_callbacks/tensorboard_root_log_dir created successfully.]
